In [131]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from tensorflow import keras

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [132]:
#데이터 불러오기
data_df = pd.read_excel('2017~2022_baseball_data.xlsx')
target_df = pd.read_excel('2017~2022_label.xlsx')

In [133]:
#훈련세트, 검증세트, 테스트세트 분리 
X_full_set, X_test=  train_test_split(data_df, test_size = 0.2, random_state = 7)
y_full_set, y_test = train_test_split(target_df, test_size = 0.2, random_state = 7)

X_train, X_valid = train_test_split(X_full_set, test_size = 0.2)
y_train, y_valid = train_test_split(y_full_set, test_size = 0.2)


In [134]:
#데이터 정규화
X_full_set=StandardScaler().fit_transform(X_full_set)
X_train=StandardScaler().fit_transform(X_train)
X_valid=StandardScaler().fit_transform(X_valid)


In [135]:
#차원축소
#pca = PCA(n_components=0.95)
#data_reduced = pca.fit_transform(data_std)


In [136]:
#모델 만들기
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[1,30]))
model.add(keras.layers.Dense(300,activation='relu'))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(2,activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer='sgd', metrics = ['accuracy'])

history = model.fit

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 30)                0         
                                                                 
 dense (Dense)               (None, 300)               9300      
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 2)                 202       
                                                                 
Total params: 39,602
Trainable params: 39,602
Non-trainable params: 0
_________________________________________________________________
